https://towardsdatascience.com/stock-prediction-using-recurrent-neural-networks-c03637437578

https://viblo.asia/p/recurrent-neural-networkphan-1-tong-quan-va-ung-dung-jvElaB4m5kw

https://nttuan8.com/bai-13-recurrent-neural-network/

https://lilianweng.github.io/lil-log/2017/07/08/predict-stock-prices-using-RNN-part-1.html

https://www.kaggle.com/raoulma/ny-stock-price-prediction-rnn-lstm-gru

In [23]:
import numpy as np
import pandas as pd
import math
import sklearn
from sklearn.preprocessing import MinMaxScaler
import datetime
import os
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

# split data in 80%/10%/10% train/validation/test sets
valid_set_size_percentage = 10 
test_set_size_percentage = 10 

# #display parent directory and working directory
# print(os.path.dirname(os.getcwd())+':', os.listdir(os.path.dirname(os.getcwd())));
# print(os.getcwd()+':', os.listdir(os.getcwd()));

In [24]:
# import all stock prices 
df = pd.read_csv("../db/IBM.csv", index_col = 0)
df.info()
df

# # number of different stocks
# print('\nnumber of different stocks: ', len(list(set(df.symbol))))
# print(list(set(df.symbol))[:10])

<class 'pandas.core.frame.DataFrame'>
Index: 2705 entries, 2010-10-11 to 2021-07-09
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       2705 non-null   float64
 1   High       2705 non-null   float64
 2   Low        2705 non-null   float64
 3   Close      2705 non-null   float64
 4   Adj Close  2705 non-null   float64
 5   Volume     2705 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 147.9+ KB


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-10-11,138.789993,139.940002,138.639999,139.660004,97.898521,4004300
2010-10-12,138.399994,139.990005,138.270004,139.850006,98.031708,5637300
2010-10-13,139.910004,141.479996,139.779999,140.369995,98.396217,8784300
2010-10-14,140.350006,141.500000,139.690002,141.500000,99.188316,5653100
2010-10-15,142.100006,142.100006,140.539993,141.059998,98.879852,7220400
...,...,...,...,...,...,...
2021-07-02,146.910004,146.949997,139.460007,140.020004,140.020004,16811200
2021-07-06,139.990005,140.419998,137.100006,138.779999,138.779999,8093700
2021-07-07,138.759995,140.330002,138.759995,139.820007,139.820007,4059700


In [25]:
df.describe()
# df.columns=["open", "high","low", "close", "adjclose", "volume"]


,Open,High,Low,Close,Adj Close,Volume
count,2705.000000,2705.000000,2705.000000,2705.000000,2705.000000,2.705000e+03
mean,159.543061,160.735464,158.404961,159.582895,127.801056,4.677263e+06
std,25.341834,25.357783,25.374706,25.370052,12.702604,2.589162e+06
min,94.599998,97.739998,90.559998,94.769997,88.795891,1.193000e+06
25%,141.119995,142.229996,140.169998,141.220001,119.876671,3.241100e+06
50%,155.960007,157.199997,154.729996,155.830002,127.041580,4.023100e+06
75%,182.199997,183.429993,180.990005,182.229996,137.951584,5.210300e+06
max,215.380005,215.899994,214.300003,215.800003,157.775299,3.806350e+07


In [26]:
# choose a specific stock
# drop feature: volume
# normalize stock data
# create train, validation and test data sets

# function to create train, validation, test data given stock data and sequence length
def load_data(stock, seq_len):
    data_raw = stock
    data = []
    
    # create all possible sequences of length seq_len
    for index in range(len(data_raw) - seq_len): 
        data.append(data_raw[index: index + seq_len])

    data = np.array(data)

    valid_set_size = int(np.round(valid_set_size_percentage/100*data.shape[0]));  
    test_set_size = int(np.round(test_set_size_percentage/100*data.shape[0]));
    train_set_size = data.shape[0] - (valid_set_size + test_set_size);
    
    x_train = data[:train_set_size,:-1,:4]
    y_train = data[:train_set_size,-1,:4]
    
    x_valid = data[train_set_size:train_set_size+valid_set_size,:-1,:4]
    y_valid = data[train_set_size:train_set_size+valid_set_size,-1,:4]
    
    x_test = data[train_set_size+valid_set_size:,:-1,:4]
    y_test = data[train_set_size+valid_set_size:,-1,:4]
    
    return [x_train, y_train, x_valid, y_valid, x_test, y_test]

# choose one stock
df_stock = df.copy()
df_stock = df_stock[['Open', 'High', 'Low', 'Close']]

cols = list(df_stock.columns.values)
print('df_stock.columns.values = ', cols)

# min-max normalization of stock
scaler = MinMaxScaler()
df_stock_norm = df_stock.copy()
df_stock_norm = df_stock_norm.values
df_stock_norm = scaler.fit_transform(df_stock_norm)

# create train, test data
seq_len = 20 # choose sequence length
x_train, y_train, x_valid, y_valid, x_test, y_test = load_data(df_stock_norm, seq_len)
print('x_train.shape = ',x_train.shape)
print('y_train.shape = ', y_train.shape)
print('x_valid.shape = ',x_valid.shape)
print('y_valid.shape = ', y_valid.shape)
print('x_test.shape = ', x_test.shape)
print('y_test.shape = ',y_test.shape)


df_stock.columns.values =  ['Open', 'High', 'Low', 'Close']
x_train.shape =  (2149, 19, 4)
y_train.shape =  (2149, 4)
x_valid.shape =  (268, 19, 4)
y_valid.shape =  (268, 4)
x_test.shape =  (268, 19, 4)
y_test.shape =  (268, 4)


In [27]:
# 4. Model and validate data 
# RNNs with basic

## Basic Cell RNN in tensorflow

index_in_epoch = 0;
perm_array  = np.arange(x_train.shape[0])
np.random.shuffle(perm_array)

# function to get the next batch
def get_next_batch(batch_size):
    global index_in_epoch, x_train, perm_array   
    start = index_in_epoch
    index_in_epoch += batch_size
    
    if index_in_epoch > x_train.shape[0]:
        np.random.shuffle(perm_array) # shuffle permutation array
        start = 0 # start next epoch
        index_in_epoch = batch_size
        
    end = index_in_epoch
    return x_train[perm_array[start:end]], y_train[perm_array[start:end]]

# parameters
n_steps = seq_len-1 
n_inputs = 4 
n_neurons = 200 
n_outputs = 4
n_layers = 2
learning_rate = 0.001
batch_size = 50
n_epochs = 100 
train_set_size = x_train.shape[0]
test_set_size = x_test.shape[0]
tf.compat.v1.reset_default_graph()
# tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_outputs])

# use Basic RNN Cell
layers = [tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons, activation=tf.nn.elu)
          for layer in range(n_layers)]

# use Basic LSTM Cell 
#layers = [tf.contrib.rnn.BasicLSTMCell(num_units=n_neurons, activation=tf.nn.elu)
#          for layer in range(n_layers)]

# use LSTM Cell with peephole connections
#layers = [tf.contrib.rnn.LSTMCell(num_units=n_neurons, 
#                                  activation=tf.nn.leaky_relu, use_peepholes = True)
#          for layer in range(n_layers)]

# use GRU cell
#layers = [tf.contrib.rnn.GRUCell(num_units=n_neurons, activation=tf.nn.leaky_relu)
#          for layer in range(n_layers)]
                                                                     
multi_layer_cell = tf.nn.rnn_cell.MultiRNNCell(layers)
rnn_outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype=tf.float32)

stacked_rnn_outputs = tf.reshape(rnn_outputs, [-1, n_neurons]) 
stacked_outputs = tf.layers.dense(stacked_rnn_outputs, n_outputs)
outputs = tf.reshape(stacked_outputs, [-1, n_steps, n_outputs])
outputs = outputs[:,n_steps-1,:] # keep only last output of sequence
                                              
loss = tf.reduce_mean(tf.square(outputs - y)) # loss function = mean squared error 
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate) 
training_op = optimizer.minimize(loss)
                                              
# run graph
with tf.Session() as sess: 
    sess.run(tf.global_variables_initializer())
    for iteration in range(int(n_epochs*train_set_size/batch_size)):
        x_batch, y_batch = get_next_batch(batch_size) # fetch the next training batch 
        
        # sess.run(training_op, feed_dict={X: x_batch.reshape(((50 , 19, 4))), y: y_batch.reshape(((, 4)))}) 
        sess.run(training_op, feed_dict={X: x_batch, y: y_batch}) 

        if iteration % int(5*train_set_size/batch_size) == 0:
            mse_train = loss.eval(feed_dict={X: x_train, y: y_train}) 
            mse_valid = loss.eval(feed_dict={X: x_valid, y: y_valid}) 
            print('%.2f epochs: MSE train/valid = %.6f/%.6f'%(
                iteration*batch_size/train_set_size, mse_train, mse_valid))

    y_train_pred = sess.run(outputs, feed_dict={X: x_train})
    y_valid_pred = sess.run(outputs, feed_dict={X: x_valid})
    y_test_pred = sess.run(outputs, feed_dict={X: x_test})
    


C:\Users\PC\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\keras\layers\legacy_rnn\rnn_cell_impl.py:427: UserWarning: `tf.nn.rnn_cell.BasicRNNCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.SimpleRNNCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.BasicRNNCell` is deprecated and will be "
C:\Users\PC\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '
C:\Users\PC\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\keras\legacy_tf_layers\core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated 

In [28]:
y_test_actual = scaler.inverse_transform(y_test)
y_test_predictions = scaler.inverse_transform(y_test_pred)
save = pd.DataFrame({'Open':y_test_actual[:,0],'High':y_test_actual[:,1], 'Low':y_test_actual[:,2],'Close':y_test_actual[:,3],'Open_Pred':y_test_predictions[:,0],'High_Pred':y_test_predictions[:,1], 'Low_Pred':y_test_predictions[:,2],'Close_Pred':y_test_predictions[:,3]})
save.index= df.index[-y_test.shape[0]-1:-1]
save.to_csv('../out/IBM_RNN_1234.csv')